In [1]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [2]:
'''cd drive/MyDrive/RoboA/'''

'cd drive/MyDrive/RoboA/'

# Markowitz Efficient Frontier

## 1. Imports

In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import statistics
from tqdm import tqdm
from collections import defaultdict
import math
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import LSTM, Dense, Dropout # type: ignore

from fetchData import fetch_raw_data_yf, getSNP500, fetch_raw_data_yf_all, getNasdaq_comp
from MonteCarloRBA import MonteCarloRBA
from PortfolioFunction import maximize_sharpe, create_correlation_matrix, get_sharpe_ratio, get_matrices
from LearningRBA import find_best_asset_to_remove, find_asset_to_add


## 2. Fetch Data

### Get all Nasdaq Stocks

In [4]:
assets= [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "AMZN",  # Amazon.com Inc.
    "GOOGL", # Alphabet Inc. (Google) Class A
    "GOOG",  # Alphabet Inc. (Google) Class C
    "META",    # Meta Platforms Inc (formerly Facebook)
    "TSLA",  # Tesla Inc
    "UA", # Berkshire Hathaway Inc. Class B
    "JPM",   # JPMorgan Chase & Co.
    "V",     # Visa Inc.
    "JNJ",   # Johnson & Johnson
    "WMT",   # Walmart Inc.
    "PG",    # Procter & Gamble Co.
    "UNH",   # UnitedHealth Group Inc.
    "MA",    # Mastercard Inc.
    "NVDA",  # NVIDIA Corporation
    "HD",    # Home Depot Inc.
    "BAC",   # Bank of America Corp
    "DIS",   # Walt Disney Co
    "PYPL",  # PayPal Holdings
    "VZ",    # Verizon Communications Inc.
    "ADBE",  # Adobe Inc.
    "CMCSA", # Comcast Corporation
    "NFLX",  # Netflix Inc.
    "KO",    # Coca-Cola Co
    "NKE",   # NIKE Inc.
    "PFE",   # Pfizer Inc.
    "MRK",   # Merck & Co., Inc.
    "PEP",   # PepsiCo, Inc.
    "T",     # AT&T Inc.
    "ABT",   # Abbott Laboratories
    "CRM",   # Salesforce.com Inc.
    "ORCL",  # Oracle Corporation
    "ABBV",  # AbbVie Inc.
    "CSCO",  # Cisco Systems, Inc.
    "INTC",  # Intel Corporation
    "TMO",   # Thermo Fisher Scientific Inc.
    "XOM",   # Exxon Mobil Corporation
    "ACN",   # Accenture plc
    "LLY",   # Eli Lilly and Company
    "COST",  # Costco Wholesale Corporation
    "MCD",   # McDonald's Corp
    "DHR",   # Danaher Corporation
    "MDT",   # Medtronic plc
    "NEE",   # NextEra Energy, Inc.
    "BMY",   # Bristol-Myers Squibb Company
    "QCOM",  # Qualcomm Inc
    "CVX",   # Chevron Corporation
    "WFC",   # Wells Fargo & Co
    "LMT",    # Lockheed Martin Corporation
    "GS",   # Goldman Sachs Group, Inc.
    "MS",   # Morgan Stanley
    "IBM",  # International Business Machines Corporation
    "GE",   # General Electric Company
    "F",    # Ford Motor Company
    "GM",   # General Motors Company
    "UBER", # Uber Technologies, Inc.
    "LYFT", # Lyft, Inc.
    "SNAP", # Snap Inc.
    "TWTR", # Twitter, Inc.
    "SPOT", # Spotify Technology S.A.
    "AMD",  # Advanced Micro Devices, Inc.
    "TXN",  # Texas Instruments Incorporated
    "BABA", # Alibaba Group Holding Limited
    "SAP",  # SAP SE
    "HON",  # Honeywell International Inc.
    "BA",   # Boeing Company
    "RTX",  # Raytheon Technologies Corporation
    "CAT",  # Caterpillar Inc.
    "DE",   # Deere & Company
    "MMM",  # 3M Company
    "DUK",  # Duke Energy Corporation
    "SO",   # Southern Company
    "EXC",  # Exelon Corporation
    "NEE",  # NextEra Energy, Inc.
    "AEP",  # American Electric Power Company, Inc.
    "SRE",  # Sempra Energy
    "ETN",  # Eaton Corporation plc
    "EMR",  # Emerson Electric Co.
    "SYY",  # Sysco Corporation
    "KR",   # Kroger Co.
    "GIS",  # General Mills, Inc.
    "K",    # Kellogg Company
    "CPB",  # Campbell Soup Company
    "MO",   # Altria Group, Inc.
    "PM",   # Philip Morris International Inc.
    "BTI",  # British American Tobacco plc
    "RDY",  # Dr. Reddy's Laboratories Ltd.
    "GILD", # Gilead Sciences, Inc.
    "BIIB", # Biogen Inc.
    "CELG", # Celgene Corporation
    "AMGN", # Amgen Inc.
    "SYK",  # Stryker Corporation
    "BSX",  # Boston Scientific Corporation
    "ISRG", # Intuitive Surgical, Inc.
    "ZBH",  # Zimmer Biomet Holdings, Inc.
    "EW",   # Edwards Lifesciences Corporation
    "RMD",  # ResMed Inc.
    "VRTX", # Vertex Pharmaceuticals Incorporated
    "REGN",  # Regeneron Pharmaceuticals, Inc.
]

#assets = getSNP500()

In [5]:
start_date = "2015-01-01"
end_date = "2018-01-01"
raw_data, asset_errors, max_combination= fetch_raw_data_yf_all(assets, start_date, end_date)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  99 of 99 completed

5 Failed downloads:
['CELG', 'TWTR']: YFTzMissingError('possibly delisted; no timezone found')
['LYFT', 'UBER', 'SPOT']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2018-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420088400, endDate = 1514782800")')


Omitted assets ( 5 ):  ['CELG', 'LYFT', 'UBER', 'TWTR', 'SPOT']
Time to fetch data: 3.97 seconds


### Split into test and train

In [6]:
split = len(raw_data.index) // 2

raw_data_train = raw_data.iloc[:split]
raw_data_test = raw_data.iloc[split:]

## 3. Mean, Volatility and Covariance

In [7]:
names, annualized_returns, cov, correlation_matrix = get_matrices(raw_data_train)
annualized_returns

Ticker
GM_Close     -0.065445
GE_Close     -0.378309
NFLX_Close    0.639346
PEP_Close     0.155561
ZBH_Close     0.076202
                ...   
EW_Close      0.508926
SAP_Close     0.307255
ISRG_Close    0.566782
DUK_Close     0.104925
LLY_Close     0.008279
Length: 94, dtype: float64

In [8]:
volatility = np.sqrt(np.diag(cov))

risk_free_rate=0
sharpe_ratios = (annualized_returns - risk_free_rate) / volatility

In [9]:
hover_texts = [
    f"{ticker} <br>Volatility: {vol:.3f} <br>Returns: {ret:.3%} <br>Sharpe Ratio: {sr:.3f}"
    for ticker, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
]

fig = go.Figure(data=go.Scatter(
    x=volatility,
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=hover_texts,
    marker=dict(color=sharpe_ratios, colorscale = 'RdBu', size=6, line=dict(width=1), colorbar=dict(title="Sharpe<br>Ratio")
    )
))

fig.update_layout(
    title='Markowitz Mean Varience Model',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Returns',
)

fig.show()

## 4.0 Monte Carlo Method

In [10]:
all_portfolios, dominant_portfolios = MonteCarloRBA(names, cov, annualized_returns, 1000)

100%|██████████| 1000/1000 [00:02<00:00, 400.13it/s]


In [11]:
print (len(dominant_portfolios) ,len(all_portfolios))

70 1000


In [12]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=[np.sqrt(p["variance"]) for p in all_portfolios],
    y=[p["return"] for p in all_portfolios],
    mode='markers',
    marker=dict(
        color=[p["sharpe"] for p in all_portfolios],
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {np.sqrt(p['variance']):.3f}<br>" +
        f"Sharpe Ratio: {p['return'] / (np.sqrt(p['variance'])):.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in all_portfolios
    ]
))

fig1.update_layout(
    xaxis=dict(title='Volatility (Standard Deviation)'),
    yaxis=dict(title='Annualised Returns'),
    title='Sample of Random Portfolios'
)

fig1.show()

In [13]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=[np.sqrt(p["variance"]) for p in dominant_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in dominant_portfolios],
    mode='markers',
    marker=dict(
        color=[p["return"] / (np.sqrt(p["variance"])) for p in dominant_portfolios],  # Sharpe Ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {np.sqrt(p['variance']):.3f}<br>" +
        f"Sharpe Ratio: {p['sharpe']:.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in dominant_portfolios
    ],
    name="Portfolios"
))

fig2.add_trace(go.Scatter(
    x=volatility,
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=[
        f"{name} <br>Volatility: {vol:.3f} <br>Returns: {ret:.3%} <br>Sharpe Ratio: {sr:.3f}"
        for name, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
    ],
    marker=dict(
        color='brown',
        size=5,
        symbol='triangle-up',  # Sets the marker shape to a triangle
        line=dict(width=1)
    ),
    name="Individual Assets"
))

fig2.update_layout(
    title='Sample of Random Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5
    )
)

fig2.show()

## 5.0 Machine Learning Method

### 5.1 Optimization Function Only

In [14]:
def MLRBA_V1(ticker, covariances, returns, num_iterations=None, risk_free_rate = 0, 
             return_power = 1, std_power = 1, return_weight=1/3, corr_weight=1/3, vol_weight= 1/3, num_assets = 8):
    
    if num_iterations is None:
        num_iterations = min(math.comb(len(ticker), num_assets), 100000)
        
    base_portfolio = np.random.choice(list(ticker), num_assets, replace=False)
    #base_portfolio = list(ticker)[:num_assets]
    highest_weighted_sharpe = -np.inf

    all_portfolios = []

    tested_assets = set()
    best_iteration = 0

    def _get_portfolio_stats (portfolio_assets, risk_free_rate = 0):
        p_asset_ret = returns.loc[portfolio_assets].values
        p_asset_var = covariances.loc[portfolio_assets, portfolio_assets].values
        best_p_weights = maximize_sharpe(p_asset_ret, p_asset_var)
        p_ret = np.dot(best_p_weights,p_asset_ret)
        p_var = np.dot(best_p_weights, p_asset_var @ best_p_weights)
        sharpe = get_sharpe_ratio(p_ret, p_var, risk_free_rate, return_power, std_power)

        return p_asset_ret, p_asset_var, sharpe, p_ret, p_var, best_p_weights

    def _update_portfolios_array(portfolios, assets, weights, p_ret, p_var):
        portfolios.append({
            "tickers": assets,
            "weights": weights,
            "return": p_ret,
            "variance": p_var,
            "sharpe": (p_ret-risk_free_rate)/np.sqrt(p_var),
        })

    curr_ret, curr_var, curr_weighted_sharpe, curr_p_return, curr_p_variance, curr_p_weights = _get_portfolio_stats(base_portfolio, risk_free_rate)
    _update_portfolios_array(all_portfolios, base_portfolio, curr_p_weights, curr_p_return, curr_p_variance)

    good_portfolios = all_portfolios.copy()
    best_portfolio = base_portfolio.copy()

    highest_weighted_sharpe = curr_weighted_sharpe
    for i in tqdm(range(num_iterations)):
        asset_to_remove = find_best_asset_to_remove(best_portfolio, curr_var, curr_ret)     #most_correlated_asset, _, _ = find_correlation_matrix(portfolio, curr_variances)
        new_portfolio = [str(asset) for asset in best_portfolio if asset != asset_to_remove]

        ranked_assets = find_asset_to_add(new_portfolio, ticker, covariances, returns, return_weight, corr_weight, vol_weight)         # Find the next best asset to add to the portfolio
        asset_to_add = ranked_assets.index[0]

        for asset in ranked_assets.index:
            if asset not in tested_assets:
                asset_to_add = asset
                break

        new_portfolio.append(asset_to_add)
        tested_assets.add(asset_to_add)

        if len(tested_assets) >= len(ticker) - num_assets:
            print("All assets have been tested")
            break

        # Substitute in and measure portfolio performance based on sharpe ratio
        new_returns, new_var, new_weighted_sharpe, new_p_return, new_p_variance, new_p_weights = _get_portfolio_stats(new_portfolio, risk_free_rate)

        _update_portfolios_array(all_portfolios, new_portfolio, new_p_weights, new_p_return, new_p_variance)

        if new_weighted_sharpe > highest_weighted_sharpe:
            highest_weighted_sharpe = new_weighted_sharpe
            best_portfolio = new_portfolio
            curr_ret, curr_var = new_returns, new_var
            best_iteration = i  # Update the best iteration

            _update_portfolios_array(good_portfolios, new_portfolio, new_p_weights, new_p_return, new_p_variance)

            tested_assets.clear()

        # If Sharpe ratio was worse, then move on to the next least correlated asset
        # If Sharpe ratio is better, set as new base portfolio, and repeat the process for num_iterations times
        # Adjust the sharpe ratio, maybe more emphasis on returns/volatility
        # Update weights to value return or corr
        # See how many iterations it takes to get here, whats a good threshold/stopping point
        # Backtesting
        # Train a model to maybe predict the sharpe ratio of a portfolio

    base_details = good_portfolios[0]
    best_details = good_portfolios[-1]

    return base_details, best_details, good_portfolios, all_portfolios, best_iteration  

base_portfolio, best_portfolio, good_portfolios, total_portfolios, best_iteration = MLRBA_V1(names, cov, annualized_returns)
base_portfolio, best_portfolio, len(total_portfolios), best_iteration

  0%|          | 423/100000 [00:03<15:22, 107.97it/s]

All assets have been tested


({'tickers': array(['DUK_Close', 'WMT_Close', 'MRK_Close', 'UA_Close', 'MDT_Close',
         'EMR_Close', 'MA_Close', 'GILD_Close'], dtype='<U11'),
  'weights': array([2.28533988e-01, 2.90607525e-01, 0.00000000e+00, 0.00000000e+00,
         6.49844643e-16, 1.18336790e-15, 4.00944659e-01, 7.99138282e-02]),
  'return': np.float64(0.2756173589715495),
  'variance': np.float64(0.005862809541963124),
  'sharpe': np.float64(3.5995952817885954)},
 {'tickers': ['WMT_Close',
   'VRTX_Close',
   'BABA_Close',
   'ISRG_Close',
   'MCD_Close',
   'KO_Close',
   'NEE_Close',
   'BA_Close'],
  'weights': array([0.11155934, 0.02641259, 0.10654921, 0.09521843, 0.14942088,
         0.2091686 , 0.23061176, 0.07105919]),
  'return': np.float64(0.4628158217845867),
  'variance': np.float64(0.003999360837242092),
  'sharpe': np.float64(7.318345395096938)},
 424,
 337)

In [15]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in total_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in total_portfolios],
    mode='markers',
    marker=dict(
        color=[p["sharpe"] for p in total_portfolios],  # Sharpe Ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {p['variance']**0.5:.3f}<br>" +
        f"Sharpe Ratio: {p['return'] / (p['variance']**0.5):.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in total_portfolios
    ],
    name="Portfolios"
))

fig.add_trace(go.Scatter(
    x=volatility,
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=[
        f"{name} <br>Volatility: {vol:.3f} <br>Returns: {ret:.3%} <br>Sharpe Ratio: {sr:.3f}"
        for name, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
    ],
    marker=dict(
        color='brown',
        size=5,
        symbol='triangle-up',  # Sets the marker shape to a triangle
        line=dict(width=1)
    ),
    name="Individual Assets"
))

fig.update_layout(
    title='Sample of Random Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5
    )
)


fig.show()

In [16]:
sharpe_ratios = [portfolio['sharpe'] for portfolio in total_portfolios]

fig = go.Figure(data=go.Scatter(x=list(range(len(sharpe_ratios))), y=sharpe_ratios, mode='lines+markers'))
fig.update_layout(title='Sharpe Ratio Over Iterations',
                  xaxis_title='Iteration',
                  yaxis_title='Sharpe Ratio',
                  )
fig.show()

### 5.2 Reinforcement Weight Training

In [17]:
def MLRBA_V2(ticker, covariances, returns, num_iterations=None, risk_free_rate = 0, 
             return_power = 1, std_power = 1, return_weight=1/3, corr_weight=1/3, vol_weight= 1/3, num_assets = 8):
    
    if num_iterations is None:
        num_iterations = min(math.comb(len(ticker), num_assets), 100000)

    base_portfolio = np.random.choice(list(ticker), num_assets, replace=False)
    #base_portfolio = list(ticker)[:num_assets]
    highest_weighted_sharpe = -np.inf
    all_portfolios = []
    tested_assets = set()
    best_iteration = 0

    learning_rate = 0.05

    def _get_portfolio_stats(portfolio_assets, risk_free_rate=0):
        p_asset_ret = returns.loc[portfolio_assets].values
        p_asset_var = covariances.loc[portfolio_assets, portfolio_assets].values
        best_p_weights = maximize_sharpe(p_asset_ret, p_asset_var)
        p_ret = np.dot(best_p_weights, p_asset_ret)
        p_var = np.dot(best_p_weights, p_asset_var @ best_p_weights)
        sharpe = get_sharpe_ratio(p_ret, p_var, risk_free_rate, return_power, std_power)
        return p_asset_ret, p_asset_var, sharpe, p_ret, p_var, best_p_weights

    def _update_portfolios_array(portfolios, assets, weights, p_ret, p_var):
        portfolios.append({
            "tickers": assets,
            "weights": weights,
            "return": p_ret,
            "variance": p_var,
            "sharpe": (p_ret - risk_free_rate) / np.sqrt(p_var),
        })

    curr_ret, curr_var, curr_weighted_sharpe, curr_p_return, curr_p_variance, curr_p_weights = _get_portfolio_stats(base_portfolio, risk_free_rate)
    _update_portfolios_array(all_portfolios, base_portfolio, curr_p_weights, curr_p_return, curr_p_variance)

    good_portfolios = all_portfolios.copy()
    best_portfolio = base_portfolio.copy()
    highest_weighted_sharpe = curr_weighted_sharpe

    improvement_threshold = 0.001

    for i in tqdm(range(num_iterations)):
        asset_to_remove = find_best_asset_to_remove(best_portfolio, curr_var, curr_ret)
        new_portfolio = [str(asset) for asset in best_portfolio if asset != asset_to_remove]

        ranked_assets = find_asset_to_add(new_portfolio, ticker, covariances, returns, return_weight, corr_weight, vol_weight)
        asset_to_add = ranked_assets.index[0]

        for asset in ranked_assets.index:
            if asset not in tested_assets:
                asset_to_add = asset
                break

        new_portfolio.append(asset_to_add)
        tested_assets.add(asset_to_add)

        if len(tested_assets) >= len(ticker) - num_assets:
            print("All assets have been tested")
            break

        new_returns, new_var, new_weighted_sharpe, new_p_return, new_p_variance, new_p_weights = _get_portfolio_stats(new_portfolio, risk_free_rate)
        _update_portfolios_array(all_portfolios, new_portfolio, new_p_weights, new_p_return, new_p_variance)

        if new_weighted_sharpe > highest_weighted_sharpe:
            improvement = new_weighted_sharpe - highest_weighted_sharpe
            highest_weighted_sharpe = new_weighted_sharpe
            best_portfolio = new_portfolio
            curr_ret, curr_var = new_returns, new_var
            best_iteration = i

            asset_return = returns.loc[asset_to_add]
            asset_vol = np.sqrt(covariances.loc[asset_to_add, asset_to_add])
            avg_return = returns.mean()
            avg_vol = np.sqrt(np.diag(covariances)).mean()

            corr_with_portfolio = correlation_matrix.loc[new_portfolio, asset_to_add].drop(asset_to_add).mean()
            avg_corr_in_portfolio = correlation_matrix.loc[new_portfolio].drop(asset_to_add, axis=1).mean().mean()
            
            # Update weights using the current learning rate
            if asset_return > avg_return:
                return_weight += learning_rate * (asset_return - avg_return) / avg_return
            else:
                return_weight -= learning_rate * (avg_return - asset_return) / avg_return

            if asset_vol < avg_vol:
                vol_weight -= learning_rate * (avg_vol - asset_vol) / avg_vol
            else:
                vol_weight += learning_rate * (asset_vol - avg_vol) / avg_vol

            if corr_with_portfolio < avg_corr_in_portfolio:
                corr_weight += learning_rate * (avg_corr_in_portfolio - corr_with_portfolio) / avg_corr_in_portfolio
            else:
                corr_weight -= learning_rate * (corr_with_portfolio - avg_corr_in_portfolio) / avg_corr_in_portfolio

            total = return_weight + corr_weight + vol_weight
            return_weight /= total
            corr_weight /= total
            vol_weight /= total

            if improvement < improvement_threshold:
                learning_rate *= 0.95
            else:
                learning_rate *= 1.01

            _update_portfolios_array(good_portfolios, new_portfolio, new_p_weights, new_p_return, new_p_variance)
            tested_assets.clear()

    base_details = good_portfolios[0]
    best_details = good_portfolios[-1]

    print("Final Weights:", return_weight, corr_weight, vol_weight)
    
    return base_details, best_details, good_portfolios, all_portfolios, best_iteration


base_portfolio, best_portfolio, good_portfolios, total_portfolios, best_iteration = MLRBA_V2(names, cov, annualized_returns)
best_portfolio, best_portfolio['sharpe'], best_iteration

  0%|          | 260/100000 [00:02<15:21, 108.29it/s]

All assets have been tested
Final Weights: 0.8852115081971558 0.06215097850909969 0.052637513293744456


({'tickers': ['XOM_Close',
   'BABA_Close',
   'ISRG_Close',
   'MCD_Close',
   'NEE_Close',
   'WMT_Close',
   'VRTX_Close',
   'KO_Close'],
  'weights': array([1.37804746e-17, 1.20818550e-01, 9.87859262e-02, 1.42119855e-01,
         2.49909158e-01, 1.13406901e-01, 2.74309259e-02, 2.47528684e-01]),
  'return': np.float64(0.44167656797321125),
  'variance': np.float64(0.0038016572926285546),
  'sharpe': np.float64(7.163378143195376)},
 np.float64(7.163378143195376),
 174)

In [18]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in total_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in total_portfolios],
    mode='markers',
    marker=dict(
        color=[p["sharpe"] for p in total_portfolios],  # Sharpe Ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.3%}<br>Volatility: {p['variance']**0.5:.3f}<br>" +
        f"Sharpe Ratio: {p['return'] / (p['variance']**0.5):.3f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.3f}" for i in range(len(p['tickers']))])
        for p in total_portfolios
    ],
    name="Portfolios"
))

fig.add_trace(go.Scatter(
    x=volatility,
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=[
        f"{name} <br>Volatility: {vol:.3f} <br>Returns: {ret:.3%} <br>Sharpe Ratio: {sr:.3f}"
        for name, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
    ],
    marker=dict(
        color='brown',
        size=5,
        symbol='triangle-up',  # Sets the marker shape to a triangle
        line=dict(width=1)
    ),
    name="Individual Assets"
))

fig.update_layout(
    title='Sample of Random Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
    legend=dict(y=5
    )
)


fig.show()

In [19]:
sharpe_ratios = [portfolio['sharpe'] for portfolio in total_portfolios]

fig = go.Figure(data=go.Scatter(x=list(range(len(sharpe_ratios))), y=sharpe_ratios, mode='lines+markers'))
fig.update_layout(title='Sharpe Ratio Over Iterations',
                  xaxis_title='Iteration',
                  yaxis_title='Sharpe Ratio',
                  )
fig.show()

In [20]:
def run_MLRBA_multiple_times(names, cov, annualized_returns, num_runs, version):
    total_good_portfolios_length = 0
    best_portfolios = []
    best_iterations = []

    for i in range(num_runs):
        if version == 'v1':
            _, best_portfolio, good_portfolios, _, best_iteration = MLRBA_V1(names, cov, annualized_returns)
        elif version == 'v2':
            _, best_portfolio, good_portfolios, _, best_iteration = MLRBA_V2(names, cov, annualized_returns)
        
        total_good_portfolios_length += len(good_portfolios)
        best_portfolios.append(best_portfolio)
        best_iterations.append(best_iteration)

    average_length = total_good_portfolios_length / num_runs
    average_iteration = statistics.fmean(best_iterations)
    std_dev_iteration = statistics.stdev(best_iterations) if num_runs > 1 else 0

    return average_length, best_portfolios, average_iteration, std_dev_iteration, best_iterations

num_runs = 1
average_good_portfolios_length_V1, _, average_iteration_V1, std_dev_V1, best_iterations_v1 = run_MLRBA_multiple_times(names, cov, annualized_returns, num_runs, 'v1')
print("Average length of good portfolios:", average_good_portfolios_length_V1, "Average iterations:", average_iteration_V1, "Std Dev:", std_dev_V1)

average_good_portfolios_length_V2, _, average_iteration_V2, std_dev_V2, best_iterations_v2 = run_MLRBA_multiple_times(names, cov, annualized_returns, num_runs, 'v2')
print("Average length of good portfolios:", average_good_portfolios_length_V2, "Average iterations:", average_iteration_V2, "Std Dev:", std_dev_V2)

  1%|          | 572/100000 [00:05<15:43, 105.36it/s]


All assets have been tested
Average length of good portfolios: 38.0 Average iterations: 486.0 Std Dev: 0


  0%|          | 181/100000 [00:01<14:29, 114.77it/s]

All assets have been tested
Final Weights: 0.938267288596755 0.05246809726940867 0.00926461413383628
Average length of good portfolios: 13.0 Average iterations: 95.0 Std Dev: 0


In [21]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        "Iterations to Reach Best Portfolio (Line Plot)",
        "Iterations to Reach Best Portfolio (Bar Plot)"
    ],)

fig.add_trace(
    go.Scatter(x=list(range(1, num_runs + 1)), y=best_iterations_v1, mode='lines', name='MLRBA V1'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=list(range(1, num_runs + 1)), y=best_iterations_v2, mode='lines', name='MLRBA V2'),
    row=1, col=1
)
fig.update_xaxes(title_text='Run Number', row=1, col=1)
fig.update_yaxes(title_text='Average Number of Iterations to Best Portfolio', row=1, col=1)

fig.add_trace(
    go.Bar(x=['MLRBA V1'], y=[average_iteration_V1], name='MLRBA V1', width=0.4,
           error_y=dict(type='data', array=[std_dev_V1], visible=True)),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=['MLRBA V2'], y=[average_iteration_V2], name='MLRBA V2', width=0.4,
           error_y=dict(type='data', array=[std_dev_V2], visible=True)),
    row=1, col=2
)
fig.update_xaxes(title_text='Run Number', row=1, col=2)
fig.update_yaxes(title_text='Average Number of Iterations to Best Portfolio', row=1, col=2)

fig.show()

## 6.0 Portfolio Prediction using LSTM

In [22]:
class PortfolioPredictor:
    def __init__(self, raw_data_train, raw_data_test, best_portfolio, n_steps=5, epochs=50, batch_size=32):
        self.raw_data_train = raw_data_train
        self.raw_data_test = raw_data_test
        self.best_portfolio = best_portfolio
        self.n_steps = n_steps
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None
        self.history = None

    def preprocess_data(self):
        # Selecting data for the tickers
        best_portfolio_data_train = self.raw_data_train[self.best_portfolio['tickers']]
        best_portfolio_data_test = self.raw_data_test[self.best_portfolio['tickers']]
        weights = np.array(self.best_portfolio['weights'])

        # Scaling the data
        scaler = MinMaxScaler(feature_range=(0, 1))
        normalized_train_data = scaler.fit_transform(best_portfolio_data_train)
        normalized_test_data = scaler.transform(best_portfolio_data_test)

        # Calculating weighted returns
        self.weighted_returns_train = np.dot(normalized_train_data, weights)
        self.weighted_returns_test = np.dot(normalized_test_data, weights)

    def create_datasets(self, data):
        X, y = [], []
        for i in range(self.n_steps, len(data)):
            X.append(data[i - self.n_steps:i, :])
            y.append(data[i, :])
        return np.array(X), np.array(y)

    def build_model(self):
        self.model = Sequential([
            LSTM(50, activation='relu', return_sequences=True),
            Dropout(0.2),
            LSTM(50, activation='relu', return_sequences=False),
            Dropout(0.2),
            Dense(1),
        ])

        def tf_weighted_mse(y_true, y_pred, alpha=0.3):
            weights = tf.exp(alpha * tf.range(tf.shape(y_true)[0], dtype=tf.float32))
            weights /= tf.reduce_sum(weights)
            squared_errors = tf.square(y_true - y_pred)
            weighted_squared_errors = weights * squared_errors
            return tf.reduce_mean(weighted_squared_errors)

        self.model.compile(optimizer='adam', loss=tf_weighted_mse)

    def train_model(self):
        X_train_weighted, y_train_weighted = self.create_datasets(self.weighted_returns_train.reshape(-1, 1))
        self.history = self.model.fit(X_train_weighted, y_train_weighted, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.1, verbose=0)

    def predict(self, predict_days=None):
        # Generate test set data
        X_test_weighted, y_test_weighted = self.create_datasets(self.weighted_returns_test.reshape(-1, 1))
        
        # Determine the number of predictions based on predict_days
        if predict_days is not None:
            # Ensure that predict_days does not exceed the length of X_test_weighted
            predict_days = min(predict_days, len(X_test_weighted))
            # Slice the last 'predict_days' entries for prediction
            X_test_to_predict = X_test_weighted[-predict_days:]
            self.predictions = self.model.predict(X_test_to_predict)
            self.y_test_weighted = y_test_weighted[-predict_days:]
        else:
            self.predictions = self.model.predict(X_test_weighted)
            self.y_test_weighted = y_test_weighted
        
        # Normalize and compute cumulative returns for the predicted period
        return self.normalize_cumulative_returns(self.predictions)
        
    def plot_loss(self):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=np.arange(1, len(self.history.history['loss'])+1), y=self.history.history['loss'], mode='lines', name='Training Loss'))
        fig.add_trace(go.Scatter(x=np.arange(1, len(self.history.history['val_loss'])+1), y=self.history.history['val_loss'], mode='lines', name='Validation Loss'))
        fig.update_layout(title='Training and Validation Loss Over Epochs',
                          xaxis_title='Epoch',
                          yaxis_title='Loss',
                          legend_title='Type of Loss')
        fig.show()

    def plot_predictions(self):
        normalized_test = self.normalize_cumulative_returns(self.y_test_weighted)
        normalized_predicted = self.normalize_cumulative_returns(self.predictions)

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=self.raw_data_test.index[-len(normalized_test):], y=normalized_test, mode='lines', name='Actual Weighted Returns'))
        fig.add_trace(go.Scatter(x=self.raw_data_test.index[-len(normalized_predicted):], y=normalized_predicted, mode='lines', name='Predicted Weighted Returns'))
        fig.update_layout(title='Actual vs Predicted Weighted Portfolio Returns',
                          xaxis_title='Date',
                          yaxis_title='Weighted Returns',
                          legend_title='Type')
        fig.show()

    def normalize_cumulative_returns(self, data):
        data_series = pd.Series(data.flatten())
        percentage_changes = data_series.pct_change()
        cumulative_returns = (1 + percentage_changes).cumprod()
        start_value = cumulative_returns.iloc[1]
        normalized_returns = (cumulative_returns / start_value) * 100
        return normalized_returns
    


In [23]:
portfolio_predictor = PortfolioPredictor(raw_data_train, raw_data_test, best_portfolio, n_steps=3)

portfolio_predictor.preprocess_data()
portfolio_predictor.build_model()
portfolio_predictor.train_model()
prediction = portfolio_predictor.predict()   
portfolio_predictor.plot_loss()
portfolio_predictor.plot_predictions()

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


In [24]:
best_sharpe = best_portfolio['sharpe']

close_to_best = []
for i in range(len(good_portfolios)):
    difference = abs((best_sharpe - good_portfolios[i]['sharpe']) / best_sharpe)
    if difference < 0.05:
        close_to_best.append(good_portfolios[i])

In [25]:
portfolio_results = {}
predict_days = 5

for id, portfolio in enumerate(close_to_best):
    portfolio_predictor = PortfolioPredictor(raw_data_train, raw_data_test, portfolio, n_steps=5, epochs=10)
    portfolio_predictor.preprocess_data()
    portfolio_predictor.build_model()
    portfolio_predictor.train_model()
    prediction = portfolio_predictor.predict(predict_days)   
    portfolio_results[id] = prediction[predict_days-1]

best_id = max(portfolio_results, key=portfolio_results.get)
predicted_best_portfolio = close_to_best[best_id]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step


In [26]:
portfolio_results, best_id, predicted_best_portfolio

({0: np.float32(99.39751),
  1: np.float32(99.46471),
  2: np.float32(100.44447),
  3: np.float32(99.91032),
  4: np.float32(99.06863),
  5: np.float32(99.23694),
  6: np.float32(99.04446)},
 2,
 {'tickers': ['XOM_Close',
   'AEP_Close',
   'BABA_Close',
   'ISRG_Close',
   'MCD_Close',
   'NEE_Close',
   'VRTX_Close',
   'WMT_Close'],
  'weights': array([0.04011113, 0.00332505, 0.13667226, 0.10815093, 0.17315699,
         0.35429648, 0.03265533, 0.15163183]),
  'return': np.float64(0.4677513733189075),
  'variance': np.float64(0.004619152461112733),
  'sharpe': np.float64(6.882305121262619)})

In [27]:
window_size = 5
windows = len(raw_data_test) // window_size

all_good_portfolios = []

for i in range(8):
    curr_split = i * window_size
    loop_raw_data_train = raw_data.iloc[:split + curr_split]
    loop_raw_data_test = raw_data.iloc[split + curr_split:]

    loop_names, loop_annualized_returns, loop_cov, _ = get_matrices(loop_raw_data_train)
    _, loop_best_portfolio, loop_good_portfolios, _, _ = MLRBA_V2(loop_names, loop_cov, loop_annualized_returns)

    best_sharpe = loop_best_portfolio['sharpe']

    close_to_best = []
    for i in range(len(good_portfolios)):
        difference = abs((best_sharpe - good_portfolios[i]['sharpe']) / best_sharpe)
        if difference < 0.05:
            close_to_best.append(good_portfolios[i])

    portfolio_results = {}
    for id, portfolio in enumerate(close_to_best):
        portfolio_predictor = PortfolioPredictor(loop_raw_data_train, loop_raw_data_test, portfolio, n_steps=window_size, epochs=10)
        portfolio_predictor.preprocess_data()
        portfolio_predictor.build_model()
        portfolio_predictor.train_model()
        prediction = portfolio_predictor.predict(window_size)   

        portfolio_results[id] = prediction[window_size-1]

    best_id = max(portfolio_results, key=portfolio_results.get)
    predicted_best_portfolio = close_to_best[best_id]
    
    start_date = loop_raw_data_test.index[0]
    end_date = loop_raw_data_test.index[window_size-1]
    all_good_portfolios.append({
        "portfolio": predicted_best_portfolio,
        "start_date": start_date,
        "end_date": end_date
    })


  0%|          | 233/100000 [00:02<15:36, 106.53it/s]


All assets have been tested
Final Weights: 0.9619867844173374 0.024109449557271463 0.013903766025391085
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step


  0%|          | 140/100000 [00:01<14:41, 113.28it/s]


All assets have been tested
Final Weights: 0.8505571257176311 0.036681623726346306 0.11276125055602268
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step


  0%|          | 161/100000 [00:01<14:35, 114.04it/s]


...

All assets have been tested
Final Weights: 0.9049812297612356 0.05117837606629989 0.04384039417246463
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step


  0%|          | 195/100000 [00:01<15:46, 105.45it/s]


All assets have been tested
Final Weights: 0.8143569111901514 0.08943504703242731 0.09620804177742126
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step


  0%|          | 269/100000 [00:02<14:10, 117.28it/s]


All assets have been tested
Final Weights: 0.8597593176469055 0.04624877394239198 0.09399190841070258
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step


In [28]:
def extract_asset_returns(raw_data, assets, start_date, end_date):
    if not isinstance(raw_data.index, pd.DatetimeIndex):
        raw_data.index = pd.to_datetime(raw_data.index)

    filtered_data = raw_data.loc[start_date:end_date, assets]

    return filtered_data

ML_portfolio = []

for i in range(len(all_good_portfolios)):
    curr_best_portfolio = all_good_portfolios[i]['portfolio']
    best_curr_port_assets = curr_best_portfolio['tickers']

    best_curr_port_assets_test_data = extract_asset_returns(raw_data, best_curr_port_assets, all_good_portfolios[i]['start_date'], all_good_portfolios[i]['end_date'])

    curr_best_portfolio_weights = curr_best_portfolio['weights']

    weighted_returns = best_curr_port_assets_test_data.mul(curr_best_portfolio_weights, axis='columns')
    portfolio_daily_returns = weighted_returns.sum(axis=1)

    ML_portfolio.append(portfolio_daily_returns)

ML_portfolio

[Date
 2017-08-01    75.706303
 2017-08-02    75.774636
 2017-08-03    75.661567
 2017-08-04    75.428653
 2017-08-07    76.512029
 dtype: float64,
 Date
 2017-08-08    76.224679
 2017-08-09    76.331773
 2017-08-10    75.501583
 2017-08-11    75.841490
 2017-08-14    76.590805
 dtype: float64,
 Date
 2017-08-15    77.320982
 2017-08-16    77.797702
 2017-08-17    77.761162
 2017-08-18    78.426757
 2017-08-21    78.803639
 dtype: float64,
 Date
 2017-08-22    80.107323
 2017-08-23    79.951164
 2017-08-24    79.822815
 2017-08-25    79.299076
 2017-08-28    79.085523
 dtype: float64,
 Date
 2017-08-29    78.946651
 2017-08-30    79.798530
 2017-08-31    80.146659
 2017-09-01    80.105697
 2017-09-05    80.017727
 dtype: float64,
 Date
 2017-09-06    79.508737
 2017-09-07    80.408703
 2017-09-08    80.089212
 2017-09-11    81.276418
 2017-09-12    80.508041
 dtype: float64,
 Date
 2017-09-13    81.009649
 2017-09-14    80.725443
 2017-09-15    80.709734
 2017-09-18    81.043291
 2017-

In [29]:
ML_portfolio_streamed = pd.concat(ML_portfolio)
ML_daily_returns = ML_portfolio_streamed.pct_change()
ML_cumulative_returns = (1 + ML_daily_returns).cumprod()

ML_cumulative_returns.iloc[0] = 1
ML_portfolio_normalized = (ML_cumulative_returns / ML_cumulative_returns.iloc[0]) * 100

Nasdaq_comp = getNasdaq_comp(ML_portfolio_streamed.index[0], ML_portfolio_streamed.index[-1])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Nasdaq_comp.index,
    y=Nasdaq_comp['Normalized'],
    mode='lines',
    name='Nasdaq Composite'
))

fig.add_trace(go.Scatter(
    x=ML_cumulative_returns.index,
    y=ML_portfolio_normalized,
    mode='lines',
    name='Portfolio Growth'
))

fig.update_layout(
    title='Comparison of Portfolio vs. Nasdaq Composite Growth',
    xaxis_title='Date',
    yaxis_title='Normalized Value (Base 100%)',
    xaxis=dict(
        type='date',
        tickformat='%b %Y',
        tickmode='auto'
    )
)

[*********************100%***********************]  1 of 1 completed

Omitted assets: []
Time to fetch data: 0.10 seconds
Max combination of assets with complete data: 1


## 7.0 Find Optimal Portfolio Size

In [30]:
all_portfolios, dominant_portfolios = MonteCarloRBA(names, cov, annualized_returns, 10000, "random", 3, 50)

100%|██████████| 10000/10000 [00:06<00:00, 1599.08it/s]


In [31]:
rd_portfolio_sizes = [len(portfolio['tickers']) for portfolio in all_portfolios]
rd_volatility = [np.sqrt(portfolio['variance']) for portfolio in all_portfolios]
rd_returns = [portfolio['return'] for portfolio in all_portfolios]

volatility_by_size = defaultdict(list)
for size, vol, ret in zip(rd_portfolio_sizes, rd_volatility, rd_returns):
    volatility_by_size[size].append((vol, ret))

average_volatility = {size: np.mean([v[0] for v in vols]) for size, vols in volatility_by_size.items()}
average_returns = {size: np.mean([v[1] for v in vols]) for size, vols in volatility_by_size.items()}

sorted_sizes = sorted(average_volatility.keys())
sorted_average_vols = [average_volatility[size] for size in sorted_sizes]
sorted_average_rets = [average_returns[size] for size in sorted_sizes]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=sorted_sizes,
    y=sorted_average_vols,
    mode='lines',
    name='Average Volatility'
))
fig.add_trace(go.Scatter(
    x=sorted_sizes,
    y=sorted_average_rets,
    mode='lines',
    name='Average Returns'
))

fig.update_layout(
    title='Average Volatility and Returns by Portfolio Size',
    xaxis_title='Number of Assets in Portfolio',
    yaxis_title='Average Value',
    xaxis=dict(type='category'),
)

fig.show()


## 8.0 Testing Against Others

In [33]:
best_port_assets = best_portfolio['tickers']
best_port_assets_test_data = raw_data_test.loc[:, best_port_assets]

Nasdaq_comp = getNasdaq_comp(best_port_assets_test_data.index[0], best_port_assets_test_data.index[-1])

best_portfolio_weights = best_portfolio['weights']
daily_returns = best_port_assets_test_data.pct_change()
weighted_returns = daily_returns.mul(best_portfolio_weights, axis='columns')
portfolio_daily_returns = weighted_returns.sum(axis=1)
portfolio_cumulative_returns = (1 + portfolio_daily_returns).cumprod()

portfolio_start = portfolio_cumulative_returns.iloc[0]
portfolio_normalized = (portfolio_cumulative_returns / portfolio_start) * 100

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Nasdaq_comp.index,
    y=Nasdaq_comp['Normalized'],
    mode='lines',
    name='Nasdaq Composite'
))

fig.add_trace(go.Scatter(
    x=portfolio_cumulative_returns.index,
    y=portfolio_normalized,
    mode='lines',
    name='Portfolio Growth'
))

fig.add_trace(go.Scatter(
    x=ML_cumulative_returns.index,
    y=ML_portfolio_normalized,
    mode='lines',
    name='Portfolio Growth'
))

fig.update_layout(
    title='Comparison of Portfolio vs. Nasdaq Composite Growth',
    xaxis_title='Date',
    yaxis_title='Normalized Value (Base 100)',
    xaxis=dict(
        type='date',
        tickformat='%b %Y',
        tickmode='auto'
    )
)

[*********************100%***********************]  1 of 1 completed

Omitted assets: []
Time to fetch data: 0.02 seconds
Max combination of assets with complete data: 1
